In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
from arrakis_nd import *
import h5py

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
file_name = "/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun4_1E19_RHC/MiniRun4_1E19_RHC.flow/FLOW/MiniRun4_1E19_RHC.flow.00000.FLOW.hdf5"

In [3]:
flow = h5py.File(file_name, 'r')

In [4]:
wvfms = flow["light"]['wvfm/data']['samples']

In [5]:
# find all peaks
wvfm_d = np.diff(wvfms, axis=-1)


In [6]:
wvfm_d.shape

(416, 8, 64, 999)

In [25]:
ntpc= 8
ndet = 64
mask = np.full(wvfms.shape, False)
# find all peaks
wvfm_d = np.diff(wvfms, axis=-1)
peaks = ((np.sign(wvfm_d[..., 1:]) * np.sign(wvfm_d[..., :-1]) < 0)
            & (np.sign(np.diff(wvfm_d, axis=-1)) <= 0)
            & ~np.isin(np.arange(ndet), mask).reshape(1, 1, -1, 1)
            & np.all(~mask, axis=-1, keepdims=True))
peaks = np.where(peaks)  # tuple of (ev, tpc, det, index)
peak_max = wvfms[..., 1:][peaks]  # waveform value at each peak

In [26]:
# Unpack the arrays from the tuple
event_ids, tpcs, detectors, ticks = peaks

# Create a boolean mask that is True where the event_id is 1, the tpc is 1, and the detector is 4
mask = (event_ids == 0) & (tpcs == 0) & (detectors == 4)

# Use the mask to index the ticks array
selected_ticks = ticks[mask]

In [27]:
import plotly.graph_objects as go

# Assuming wvfms is your waveform data and peak_indices contains the indices of the peaks
waveform = wvfms[0,0,4,:]  # replace with the index of the waveform you want to plot


fig = go.Figure()

# Plot the waveform
fig.add_trace(go.Scatter(
    y=waveform,
    mode='lines',
    name='Waveform'
))

# Plot the peaks
fig.add_trace(go.Scatter(
    x=selected_ticks+1,
    y=waveform[selected_ticks+1],
    mode='markers',
    name='Peaks'
))

fig.show()

In [29]:
# convert channel thresholds into an array
threshold_array = np.zeros((ntpc, ndet, 1))
for tpc in range(ntpc):
    for det in range(ndet):
        threshold_array[tpc,
                        det] = 500
threshold = threshold_array

In [31]:
# apply threshold
threshold_mask = peak_max >= threshold[peaks[1:-1]].ravel()

In [32]:
if np.count_nonzero(threshold_mask):
    # hits are present in event, extract parameters
    peaks = tuple(p[threshold_mask].reshape(-1, 1) for p in peaks)
    peak_max = peak_max[threshold_mask]

In [34]:
# get neighboring samples
near_samples = 3
nsamples = wvfms.shape[0]
peak_sample_index = np.clip(peaks[-1].reshape(-1, 1)
                            + np.arange(-near_samples + 1, near_samples + 2), 0, nsamples - 1)
peak_samples = wvfms[peaks[:-1] + (peak_sample_index,)]
peak_sum = np.sum(peak_samples.astype(int), axis=-1)

In [36]:
# create hit spline
import scipy
peak_spline = scipy.interpolate.CubicSpline(
    np.arange(-near_samples, near_samples + 1),
    peak_samples, axis=-1, extrapolate=True)
# calculate integral
peak_sum_spline = peak_spline.integrate(-near_samples,
                                        near_samples)

In [39]:
# find max
interpolation = 256
sample_rate = 1/16
subsamples = np.linspace(-near_samples, near_samples,
                            interpolation)
peak_spline_subsamples = peak_spline(subsamples)
peak_max_spline = np.max(peak_spline_subsamples, axis=-1)
peak_ns_spline = np.expand_dims(np.take_along_axis(subsamples,
                                                    np.argmax(peak_spline_subsamples, axis=-1), axis=0), axis=-1) * sample_rate


In [44]:
def find_outlier_mask(arr):
    '''
        Find outlier mask using median absolute deviation. An outlier is
        defined as::

            |arr - median(arr, axis=-1)| >
                median(|arr - median(arr, axis=-1)|, axis=-1)

        :param arr: 2D masked array of points, ``shape: (N,M)``

        :returns: 2D boolean masked array of outliers, ``shape: (N,M)``, ``True == outlier``

    '''
    med = np.median(arr, axis=-1, keepdims=True)
    mad = np.median(np.abs(arr - med), axis=-1, keepdims=True)
    return np.abs(arr - med) > mad

In [45]:
# project back to 0-crossing
peak_spline_d = peak_spline.derivative(1)(subsamples)
peak_rising_spline_samples = np.array(subsamples - peak_spline_subsamples / peak_spline_d)[subsamples >= peak_ns_spline]
rising_outlier_mask = find_outlier_mask(
    peak_rising_spline_samples)
# calculate rising edge
peak_rising_spline_samples = np.array(peak_rising_spline_samples)[rising_outlier_mask]
peak_rising_spline = np.mean(peak_rising_spline_samples, axis=-1,
                                keepdims=True) * sample_rate
peak_rising_err_spline = np.std(peak_rising_spline_samples, axis=-1,
                                keepdims=True) * sample_rate

/tmp/ipykernel_1647857/1995876067.py:3: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_1647857/1995876067.py:3: RuntimeWarning:

invalid value encountered in divide

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning:

invalid value encountered in divide

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in divide

/global/homes/m/mnuland/.conda/envs/arrakis_env/lib/python3.11/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in divide



In [52]:
hits_dtype = np.dtype([
            ('id', 'u4'),
            ('tpc', 'u1'),
            ('det', 'u1'),
            ('sample_idx', 'u2'),
            ('ns', 'f8'),
            ('busy_ns', 'f8'),
            ('samples', 'f4', (2 * near_samples + 1,)),
            ('sum', 'f4'),
            ('max', 'f4'),
            ('sum_spline', 'f4'),
            ('max_spline', 'f4'),
            ('ns_spline', 'f4'),
            ('rising_spline', 'f4'),
            ('rising_err_spline', 'f4'),
            ('fwhm_spline', 'f4')
        ])
wvfm_det = np.broadcast_to(np.arange(wvfms.shape[-2]).reshape(1,1,-1), wvfms.shape[:-1])
wvfm_align = cache[self.wvfm_align_dset_name].reshape(cache[source_name].shape)

In [54]:
# calculate FWHM
peak_lhm_spline_samples = np.array(subsamples
                                    + (np.expand_dims(peak_max_spline, axis=-1) * 0.5 - peak_spline_subsamples) / peak_spline_d)[subsamples >= peak_ns_spline]
peak_uhm_spline_samples = np.array(subsamples
                                    + (np.expand_dims(peak_max_spline, axis=-1) * 0.5 - peak_spline_subsamples) / peak_spline_d)[subsamples <= peak_ns_spline]
lhm_outlier_mask = find_outlier_mask(peak_lhm_spline_samples)
uhm_outlier_mask = find_outlier_mask(peak_uhm_spline_samples)
# calculate fwhm
peak_lhm_spline_samples = np.array(peak_lhm_spline_samples)[lhm_outlier_mask]
peak_uhm_spline_samples = np.array(peak_uhm_spline_samples)[uhm_outlier_mask]
peak_fwhm_spline = (peak_uhm_spline_samples.mean(axis=-1)
    - peak_lhm_spline_samples.mean(axis=-1))

hit_data = np.empty((len(peaks[-1])), dtype=hits_dtype)
hit_data['tpc'] = peaks[1].ravel()
hit_data['det'] = wvfm_det[peaks[:3]].ravel()
# hit_data['ns'] = wvfm_align['ns'][peaks[0]].ravel()
hit_data['sample_idx'] = peaks[-1].ravel() + 1

/tmp/ipykernel_1647857/3479042095.py:3: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_1647857/3479042095.py:3: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_1647857/3479042095.py:5: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_1647857/3479042095.py:5: RuntimeWarning:

invalid value encountered in divide

/tmp/ipykernel_1647857/3479042095.py:11: RuntimeWarning:

Mean of empty slice.

/tmp/ipykernel_1647857/3479042095.py:12: RuntimeWarning:

Mean of empty slice.



In [ ]:
wvfm_align = cache[self.wvfm_align_dset_name].reshape(cache[source_name].shape)

In [56]:
# align_sample_idx = wvfm_align['sample_idx']
# if align_sample_idx.ndim == 2:
#     target_shape = wvfms.shape[:-1] #(n_batch, n_tpc, n_ch)
#     n_ch = target_shape[-1]
#     align_sample_idx = np.reshape(
#         np.repeat(align_sample_idx, n_ch), target_shape
#     )

# hit_data['busy_ns'] = (
#     (peaks[-1] + 1 - align_sample_idx[peaks[:3]]).ravel() 
#     * sample_rate
# )
hit_data['samples'] = peak_samples.reshape(-1, 2 * near_samples + 1)
hit_data['sum'] = peak_sum.ravel()
hit_data['max'] = peak_max.ravel()
hit_data['sum_spline'] = peak_sum_spline.ravel()
hit_data['max_spline'] = peak_max_spline.ravel()
hit_data['ns_spline'] = peak_ns_spline.ravel()
hit_data['rising_spline'] = peak_rising_spline.ravel()
hit_data['rising_err_spline'] = peak_rising_err_spline.ravel()
hit_data['fwhm_spline'] = peak_fwhm_spline.ravel()

In [57]:
hit_data['samples']

array([[ 36., 364., 600., ..., 684., 680., 600.],
       [  4.,   4.,   4., ...,   4.,   4.,   4.],
       [  4.,   4.,   4., ...,   4.,   4.,   4.],
       ...,
       [124., 404., 528., ..., 480., 448., 352.],
       [288., 484., 556., ..., 540., 456., 316.],
       [244., 504., 560., ..., 552., 488., 480.]], dtype=float32)